Orientación Rostro
Nuestro sistema de registro de documentos de usuarios está presentando una falla y
nuestros expertos han dado con el problema. Para solucionarlo requerimos de la ayuda de
un profesional en computer vision que intente resolver el siguiente requerimiento: SIN
utilizar modelos de Machine Learning, construir un sistema que dada la fotografía de un
cliente determine de forma automática la orientación del rostro dentro de la imagen, como
se muestra en la imagen.

Puedes utilizar procedimientos de visión por computadora que no involucren algoritmos de
aprendizaje automático.

# Primeiro Teste

In [14]:
import cv2
import numpy as np

def detectar_orientacao(imagem_path):
    # Carrega imagem e converte para escala de cinza
    img = cv2.imread(imagem_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 80, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    contornos, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contornos:
        return "Nenhum contorno encontrado."

    contorno = max(contornos, key=cv2.contourArea)

    if cv2.contourArea(contorno) < 1000:
        return "Rosto não identificado."

    # Encontra retângulo com ângulo mínimo ao redor do contorno
    rect = cv2.minAreaRect(contorno)
    _, (w, h), angle = rect

    # Ajuste do ângulo
    if w < h:
        angle = -angle
    else:
        angle = 90 - angle

    # Normaliza ângulo para 0 a 360
    angle = angle % 360

    # Classifica em 0, 90, 180 ou 270 (tolerância de 20 graus)
    if (angle >= 340 or angle <= 20):
        orientacao = "0° (reto)"
    elif 70 <= angle <= 110:
        orientacao = "270° (virado à esquerda)"
    elif 160 <= angle <= 200:
        orientacao = "180° (de cabeça para baixo)"
    elif 250 <= angle <= 290:
        orientacao = "90° (virado à direita)"
    else:
        orientacao = f"Ângulo intermediário: {int(angle)}° (fora das 4 orientações principais)"

    return f"Orientação detectada: {orientacao}"

In [15]:

imagem_path = "../data/zero.png"
resultado = detectar_orientacao(imagem_path)
print(resultado)

Orientação detectada: 0° (reto)


Esse teste não funcionou para os ângulos de 180 e 270, imagino que algo com a região do cabelo pode estar dando influenciando. 

# Segundo Teste - debug primeiro

In [16]:
import cv2
import numpy as np

def detectar_orientacao_debug(imagem_path):
    img = cv2.imread(imagem_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 80, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    contornos, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contornos:
        return "Nenhum contorno encontrado."

    contorno = max(contornos, key=cv2.contourArea)
    if cv2.contourArea(contorno) < 1000:
        return "Rosto não identificado."

    rect = cv2.minAreaRect(contorno)
    (x, y), (w, h), angle = rect

    # DEBUG: Print raw values
    print(f"Raw angle: {angle:.2f}, width: {w:.2f}, height: {h:.2f}")

    # Fix angle
    if w < h:
        angle = -angle
    else:
        angle = 90 - angle

    angle = angle % 360

    # Classify into 4 main orientations
    if (angle >= 340 or angle <= 20):
        orientacao = "0° (reto)"
    elif 70 <= angle <= 110:
        orientacao = "270° (virado à esquerda)"
    elif 160 <= angle <= 200:
        orientacao = "180° (de cabeça para baixo)"
    elif 250 <= angle <= 290:
        orientacao = "90° (virado à direita)"
    else:
        orientacao = f"Ângulo intermediário: {int(angle)}° (fora das 4 orientações principais)"

    print(f"Adjusted angle: {int(angle)}° -> {orientacao}")
    return f"Orientação detectada: {orientacao}"


In [18]:

imagem_path = "../data/270.png"
resultado = detectar_orientacao_debug(imagem_path)
print(resultado)

Raw angle: 90.00, width: 91.00, height: 113.00
Adjusted angle: 270° -> 90° (virado à direita)
Orientação detectada: 90° (virado à direita)


# Terceiro Teste - ajuste de ângulo

In [19]:
import cv2
import numpy as np

def ajustar_angulo(angle, w, h):
    if w < h:
        adjusted = -angle
    else:
        adjusted = -(angle + 90)
    adjusted = adjusted % 360

    print(f"Adjusted angle: {adjusted:.2f}°")

    if adjusted <= 20 or adjusted >= 340:
        return "0° (reto)"
    elif 70 <= adjusted <= 110:
        return "90° (virado à direita)"
    elif 160 <= adjusted <= 200:
        return "180° (de cabeça para baixo)"
    elif 250 <= adjusted <= 290:
        return "270° (virado à esquerda)"
    else:
        return f"Ângulo intermediário: {int(adjusted)}°"

def detectar_orientacao_final(imagem_path):
    img = cv2.imread(imagem_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 80, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    contornos, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contornos:
        return "Nenhum contorno encontrado."

    contorno = max(contornos, key=cv2.contourArea)
    if cv2.contourArea(contorno) < 1000:
        return "Rosto não identificado."

    rect = cv2.minAreaRect(contorno)
    (x, y), (w, h), angle = rect
    print(f"Raw angle: {angle:.2f}, width: {w:.2f}, height: {h:.2f}")

    orientacao = ajustar_angulo(angle, w, h)
    return f"Orientação detectada: {orientacao}"


In [25]:

imagem_path = "../data/90.png"
resultado = detectar_orientacao_final(imagem_path)
print(resultado)

Raw angle: 90.00, width: 91.00, height: 112.00
Adjusted angle: 270.00°
Orientação detectada: 270° (virado à esquerda)


# Quarta Versão

In [29]:
import cv2
import numpy as np

def ajustar_angulo(angle, w, h):
    if w < h:
        adjusted = -angle
    else:
        adjusted = -(angle + 90)
    adjusted = adjusted % 360

    print(f"Adjusted angle: {adjusted:.2f}°")

    if adjusted <= 20 or adjusted >= 340:
        return "0° (reto)"
    elif 70 <= adjusted <= 110:
        return "90° (virado à direita)"
    elif 160 <= adjusted <= 200:
        return "180° (de cabeça para baixo)"
    elif 250 <= adjusted <= 290:
        return "270° (virado à esquerda)"
    else:
        return f"Ângulo intermediário: {int(adjusted)}°"

def detectar_orientacao_v4_ajustado(imagem_path):
    img = cv2.imread(imagem_path)
    if img is None:
        return "Imagem não encontrada."

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 30, 100)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    edges_dilated = cv2.dilate(edges, kernel, iterations=1)

    contornos, _ = cv2.findContours(edges_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contornos:
        return "Nenhum contorno encontrado."

    contorno = max(contornos, key=cv2.contourArea)
    if cv2.contourArea(contorno) < 1000:
        return "Rosto não identificado."

    rect = cv2.minAreaRect(contorno)
    (x, y), (w, h), angle = rect
    print(f"Raw angle: {angle:.2f}, width: {w:.2f}, height: {h:.2f}")

    orientacao = ajustar_angulo(angle, w, h)
    return f"Orientação detectada: {orientacao}"

In [33]:
print(detectar_orientacao_v4_ajustado("../data/180.png"))

Raw angle: 90.00, width: 117.00, height: 95.00
Adjusted angle: 180.00°
Orientação detectada: 180° (de cabeça para baixo)


# Conclusões

1. Versão 1 – Thresholding + minAreaRect

Princípio: Usa limiarização simples para segmentar o rosto e encontra o menor retângulo que envolve o maior contorno.

Ponto forte: Simplicidade e rapidez; funciona razoavelmente em imagens com fundo neutro e boa iluminação.

Limitações: Sensível à iluminação, sombra, cabelo, e ruídos. A lógica de ajuste do ângulo causa erros frequentes, especialmente confunde rotações de 90° e 270°.

Robustez: Baixa. Muitas falhas em imagens rotacionadas, principalmente com variações naturais no cabelo ou fundo.

2. Versão 2 – Mesma abordagem com Debug

Princípio: Igual à versão 1, mas com impressão dos valores do ângulo e dimensões para facilitar o diagnóstico.

Ponto forte: Ajuda a entender porque a classificação falha, fornecendo dados para ajustar o cálculo.

Limitações: Lógica ainda igual à versão 1, então erros de classificação persistem.

Robustez: Não melhorou em relação à versão 1; a vantagem é só no debug.

3. Versão 3 – Ajuste refinado do ângulo

Princípio: Mesma base, mas melhora o cálculo do ângulo para normalizar corretamente e distinguir entre as rotações 90° e 270°.

Ponto forte: Corrige o problema clássico da inversão do ângulo do minAreaRect.

Limitações: Continua dependente da qualidade da segmentação via threshold, ainda sensível a ruído, sombras e cabelo, o que afeta a qualidade do contorno.

Robustez: Melhor que as anteriores, mas ainda não totalmente confiável em imagens complexas.

4. Versão 4 – Canny + minAreaRect

Princípio: Usa detecção de bordas Canny para extrair contornos, substituindo a segmentação por threshold.

Ponto forte: Captura bordas mais relevantes (traços do rosto) e é menos afetada por variações na iluminação comparada ao threshold.

Limitações: Ainda depende de parâmetros finos do Canny, pode gerar ruído em fundos complexos e falha em imagens com bordas fracas ou desfocadas. A necessidade de dilatação e ajuste fino complica a confiabilidade.

Robustez: Teoricamente melhor, mas na prática depende muito da qualidade da imagem e ajuste dos parâmetros. Ainda não perfeito para todos os casos.